In [1]:
#import libs
import torch
import os
from ModelZoo import TransformerWithAttentionOutputted
from MyDataset import CustomTimeSeriesDataset
from torch.utils.data import DataLoader

#load our transformer model weights
#Define model to be trained
name = "TransformerWithAttentionOutputted"
model = TransformerWithAttentionOutputted()
nettype = 'Transformer'
model.load_state_dict(torch.load(os.path.join('Model_Parameters', f'{name}_best_parameters.pth')))
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_test = CustomTimeSeriesDataset('ptbdb_test.csv', NetType=nettype)
test_loader = DataLoader(dataset_test, batch_size=8, shuffle=False)

#Evaluate model
model.to(device)
model.eval()  # Set best model to evaluation mode
with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass and prepare for plot
        outputs, attention_weights = model(inputs).detach().cpu().numpy()
        labels=labels.detach().cpu().numpy()


RuntimeError: Error(s) in loading state_dict for TransformerWithAttentionOutputted:
	Missing key(s) in state_dict: "positional.pe". 
	Unexpected key(s) in state_dict: "conv1.weight", "conv1.bias", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn1.num_batches_tracked", "conv2.weight", "conv2.bias", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn2.num_batches_tracked", "conv3.weight", "conv3.bias", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var", "bn3.num_batches_tracked". 
	size mismatch for TransformerEncoder.layers.0.self_attn.in_proj_weight: copying a param with shape torch.Size([1152, 384]) from checkpoint, the shape in current model is torch.Size([6, 2]).
	size mismatch for TransformerEncoder.layers.0.self_attn.in_proj_bias: copying a param with shape torch.Size([1152]) from checkpoint, the shape in current model is torch.Size([6]).
	size mismatch for TransformerEncoder.layers.0.self_attn.out_proj.weight: copying a param with